In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time
!pip install torchsummary

from torchsummary import summary
from tqdm import tqdm
import tensorflow
import tensorflow as tf
from sklearn.metrics import accuracy_score
from torchvision import models




In [2]:
# Implementation of Alexnet

class Conv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(Conv, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,
                              stride=stride, padding=padding)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.lrn = nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.lrn(x)
        return x

class AlexNetSmall(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNetSmall, self).__init__()
        self.conv1 = Conv(in_channels=3, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.conv2 = Conv(in_channels=64, out_channels=64, kernel_size=5, stride=1, padding=2)
        # two fully connected layers
        self.fc1 = nn.Linear(64 * 7 * 7, 384)
        self.fc2 = nn.Linear(384, 192)
        # 10-way linear layer is used for prediction
        self.fc3 = nn.Linear(192, num_classes)
        self.relu = nn.ReLU()

              



    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x
    

model = AlexNetSmall()
summary(model,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           4,864
              ReLU-2           [-1, 64, 32, 32]               0
         MaxPool2d-3           [-1, 64, 15, 15]               0
 LocalResponseNorm-4           [-1, 64, 15, 15]               0
              Conv-5           [-1, 64, 15, 15]               0
            Conv2d-6           [-1, 64, 15, 15]         102,464
              ReLU-7           [-1, 64, 15, 15]               0
         MaxPool2d-8             [-1, 64, 7, 7]               0
 LocalResponseNorm-9             [-1, 64, 7, 7]               0
             Conv-10             [-1, 64, 7, 7]               0
           Linear-11                  [-1, 384]       1,204,608
             ReLU-12                  [-1, 384]               0
           Linear-13                  [-1, 192]          73,920
             ReLU-14                  [

In [3]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameter
batch_size = 64
#  CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(),transforms.CenterCrop(28),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./', train=False, transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to ./
